In [17]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [19]:
train_dir = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset'  
test_dir = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset'

In [20]:
input_shape = (128, 128, 3)
batch_size = 32

In [21]:
data_generator = ImageDataGenerator(rescale=1./255)

In [22]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 6400 images belonging to 2 classes.


In [23]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 6400 images belonging to 2 classes.


In [24]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

  5/200 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step 

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step


In [26]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step


In [27]:
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 0.5004881	total: 6.81ms	remaining: 6.8s
1:	learn: 0.3675497	total: 13.7ms	remaining: 6.82s
2:	learn: 0.2955850	total: 20.2ms	remaining: 6.7s
3:	learn: 0.2470286	total: 26.7ms	remaining: 6.65s
4:	learn: 0.2126703	total: 34.5ms	remaining: 6.87s
5:	learn: 0.1867477	total: 40.4ms	remaining: 6.7s
6:	learn: 0.1647498	total: 45.9ms	remaining: 6.51s
7:	learn: 0.1489173	total: 51.3ms	remaining: 6.36s
8:	learn: 0.1359606	total: 56.2ms	remaining: 6.19s
9:	learn: 0.1289478	total: 60.8ms	remaining: 6.02s
10:	learn: 0.1222489	total: 65.7ms	remaining: 5.9s
11:	learn: 0.1140144	total: 70.7ms	remaining: 5.82s
12:	learn: 0.1097854	total: 75.3ms	remaining: 5.72s
13:	learn: 0.1066181	total: 80.2ms	remaining: 5.65s
14:	learn: 0.1025620	total: 85.1ms	remaining: 5.59s
15:	learn: 0.1003786	total: 89.7ms	remaining: 5.52s
16:	learn: 0.0969656	total: 94.8ms	remaining: 5.48s
17:	learn: 0.0924140	total: 99.8ms	remaining: 5.44s
18:	learn: 0.0896224	total: 104ms	remaining: 5.39s
19:	learn: 0.0871144	total:

In [28]:
test_predictions = cb_classifier.predict(test_features)

In [29]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 100.00%


In [30]:
model.save('cnn_feature_extractor6.h5')

In [31]:
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [32]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  

def classify_image(img_path):

    img = image.load_img(img_path, target_size=(128, 128))  
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0  

    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1) 

    prediction = catboost_classifier.predict(features)
    
    predicted_class = int(prediction[0])

    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

img_path = r'C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test/MildDemented/30 (4).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
The image is classified as: MildDemented
